# Imports and setting up SQL connection

In [1]:
import psycopg2 as psql
import pandas as pd
import numpy as np
import pandas.io.sql as sqlio
from plotnine import *
import re

In [2]:
try:
    conn = psql.connect("dbname=swhgd-popular-3k-python user=postgres password=postgres")
except psql.Error as e:
    print(type(e))
    print(e)

# SQL Queries
Here we will get all the data we need from the server using various SQL queries.  
Below is the schema for the database we will be querying.

![](dataset-schema.png)

And then here are our queries

In [3]:
query = """SELECT DISTINCT ON (o.url) o.url as url, 
o.type AS host_type,
CASE WHEN COUNT(rel.date) >= 1 THEN 1 ELSE 0 END AS major_releases
FROM revision rev 
FULL JOIN release rel on rev.id = rel.target
FULL JOIN snapshot_branch s_b ON rev.id = s_b.target 
FULL JOIN snapshot_branches s_bs ON s_bs.branch_id = s_b.object_id
FULL JOIN snapshot s ON s.object_id = s_bs.snapshot_id
FULL JOIN origin_visit ov ON ov.snapshot_id = s.object_id
FULL JOIN origin o on o.id = ov.origin
WHERE (rev.date >= '2005-01-01'::DATE AND rev.date <= '2018-01-01'::DATE)
GROUP BY o.url, rev.date, o.type"""
d = sqlio.read_sql_query(query, conn)
d

,url,host_type,major_releases
0,deb://Debian/packages/accerciser,deb,0
1,deb://Debian/packages/agtl,deb,0
2,deb://Debian/packages/ajaxterm,deb,0
3,deb://Debian/packages/alacarte,deb,0
4,deb://Debian/packages/angrydd,deb,0
...,...,...,...
2081,https://pypi.org/project/zc.lockfile/,pypi,0
2082,https://pypi.org/project/zope.component/,pypi,0
2083,https://pypi.org/project/zope.deprecation/,pypi,0
2084,https://pypi.org/project/zope.interface/,pypi,0


In [4]:
query = """
SELECT o.url, COUNT(DISTINCT rev.author) as author_count
FROM revision rev 
FULL JOIN release rel on rev.id = rel.target
FULL JOIN snapshot_branch s_b ON rev.id = s_b.target 
FULL JOIN snapshot_branches s_bs ON s_bs.branch_id = s_b.object_id
FULL JOIN snapshot s ON s.object_id = s_bs.snapshot_id
FULL JOIN origin_visit ov ON ov.snapshot_id = s.object_id
FULL JOIN origin o on o.id = ov.origin
WHERE (rev.date >= '2005-01-01'::DATE AND rev.date <= '2018-01-01'::DATE)
GROUP BY o.url;
"""
author_count = sqlio.read_sql_query(query, conn)
author_count

,url,author_count
0,deb://Debian/packages/accerciser,4
1,deb://Debian/packages/agtl,2
2,deb://Debian/packages/ajaxterm,2
3,deb://Debian/packages/alacarte,3
4,deb://Debian/packages/angrydd,3
...,...,...
2081,https://pypi.org/project/zc.lockfile/,2
2082,https://pypi.org/project/zope.component/,3
2083,https://pypi.org/project/zope.deprecation/,2
2084,https://pypi.org/project/zope.interface/,3


In [5]:
query = """SELECT o.url AS url, MIN(r.date) AS min_date
FROM revision r 
FULL JOIN snapshot_branch s_b ON r.id = s_b.target 
FULL JOIN snapshot_branches s_bs ON s_bs.branch_id = s_b.object_id
FULL JOIN snapshot s ON s.object_id = s_bs.snapshot_id
FULL JOIN origin_visit ov ON ov.snapshot_id = s.object_id
FULL JOIN origin o on o.id = ov.origin
WHERE (r.date >= '2005-01-01'::DATE AND r.date <= '2018-01-01'::DATE)
GROUP BY o.url;"""
min_dates = sqlio.read_sql_query(query, conn)
min_dates

,url,min_date
0,deb://Debian/packages/accerciser,2012-04-18 10:06:21-07:00
1,deb://Debian/packages/agtl,2011-01-22 04:55:12-08:00
2,deb://Debian/packages/ajaxterm,2011-08-19 07:31:06-07:00
3,deb://Debian/packages/alacarte,2012-06-29 03:43:50-07:00
4,deb://Debian/packages/angrydd,2008-10-19 05:19:40-07:00
...,...,...
2081,https://pypi.org/project/zc.lockfile/,2007-07-18 04:27:21-07:00
2082,https://pypi.org/project/zope.component/,2007-02-19 06:16:09-08:00
2083,https://pypi.org/project/zope.deprecation/,2007-02-18 13:55:54-08:00
2084,https://pypi.org/project/zope.interface/,2007-01-03 08:19:54-08:00


In [6]:
query = """SELECT o.url AS url, MAX(r.date) AS max_date
FROM revision r 
FULL JOIN snapshot_branch s_b ON r.id = s_b.target 
FULL JOIN snapshot_branches s_bs ON s_bs.branch_id = s_b.object_id
FULL JOIN snapshot s ON s.object_id = s_bs.snapshot_id
FULL JOIN origin_visit ov ON ov.snapshot_id = s.object_id
FULL JOIN origin o on o.id = ov.origin
WHERE (r.date >= '2005-01-01'::DATE AND r.date <= '2018-01-01'::DATE)
GROUP BY o.url;"""
max_dates = sqlio.read_sql_query(query, conn)
max_dates

,url,max_date
0,deb://Debian/packages/accerciser,2017-12-13 05:26:58-08:00
1,deb://Debian/packages/agtl,2015-08-19 13:36:54-07:00
2,deb://Debian/packages/ajaxterm,2015-05-31 10:43:09-07:00
3,deb://Debian/packages/alacarte,2017-12-18 18:44:02-08:00
4,deb://Debian/packages/angrydd,2016-04-24 12:35:20-07:00
...,...,...
2081,https://pypi.org/project/zc.lockfile/,2016-06-19 09:27:26-07:00
2082,https://pypi.org/project/zope.component/,2017-09-26 04:17:58-07:00
2083,https://pypi.org/project/zope.deprecation/,2017-08-07 12:24:01-07:00
2084,https://pypi.org/project/zope.interface/,2017-09-22 00:46:20-07:00


In [7]:
query = """SELECT o.url as url, COUNT(r.date) as rev_count
FROM revision r
FULL JOIN snapshot_branch s_b ON r.id = s_b.target 
FULL JOIN snapshot_branches s_bs ON s_bs.branch_id = s_b.object_id
FULL JOIN snapshot s ON s.object_id = s_bs.snapshot_id
FULL JOIN origin_visit ov ON ov.snapshot_id = s.object_id
FULL JOIN origin o on o.id = ov.origin
WHERE (r.date >= '2017-11-01'::DATE)
GROUP BY o.url;"""
rev_counts = sqlio.read_sql_query(query, conn)
rev_counts

,url,rev_count
0,deb://Debian/packages/accerciser,358
1,deb://Debian/packages/alacarte,347
2,deb://Debian/packages/anki,192
3,deb://Debian/packages/ansible,496
4,deb://Debian/packages/apt-listchanges,337
...,...,...
1496,https://pypi.org/project/yamllint/,3
1497,https://pypi.org/project/youtube_dl/,53
1498,https://pypi.org/project/zc.lockfile/,1
1499,https://pypi.org/project/zope.interface/,1


# Time to join the various dataframes together and do some cleaning in Python

In [8]:
durations = pd.merge(left=max_dates, right=min_dates, left_on='url', right_on='url')
durations.to_csv('max-min-dates.csv')
durations['duration'] = (((durations['max_date'] - durations['min_date'])/np.timedelta64(1,'D'))/28).astype(int) #convert timedelta in days to int that represents months
durations = durations.drop(columns=['max_date', 'min_date'])
durations

,url,duration
0,deb://Debian/packages/accerciser,73
1,deb://Debian/packages/agtl,59
2,deb://Debian/packages/ajaxterm,49
3,deb://Debian/packages/alacarte,71
4,deb://Debian/packages/angrydd,98
...,...,...
2081,https://pypi.org/project/zc.lockfile/,116
2082,https://pypi.org/project/zope.component/,138
2083,https://pypi.org/project/zope.deprecation/,136
2084,https://pypi.org/project/zope.interface/,139


Lets save the dataframes to csvs before we merge them

In [9]:
author_count.to_csv('author_count.csv')
rev_counts.to_csv('rev_counts.csv')
d.to_csv('basic-data.csv')

In [10]:
data = d.merge(author_count, on='url', how='outer').merge(durations, on='url', how='outer').merge(rev_counts, on='url', how='outer')
data = data.dropna(subset=['host_type', 'major_releases', 'author_count', 'duration'])

Lets get the project name now

In [11]:
def get_project(url):
    if(url is None):
        return
    else:
        groups = re.findall(".+/(.+)/*$", str(url))
        return groups[0].replace("/", "")
        
data['project'] = data['url'].apply(get_project)


Set some columns as string types

In [12]:
data['url'] = data['url'].astype(str)
data['host_type'] = data['host_type'].astype(str)
data['project'] = data['project'].astype(str)


Time to censor the data now

In [13]:
def get_status(rev_count):
    # the rev_count column is the count of revisions which are greater than 2017-11-01
    # choice of status variable based on Calculating survival using Excel sheets
    # from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4062335/
    if(rev_count >= 1): # if rev_count is >= 1 then the project is active (status = 0)
        status = 0
    else: # else the project is inactive (status = 1)
        status = 1
    return status
data['status'] = data['rev_count'].apply(get_status)

In [14]:
# These are mistakes the project and host_type should uniquely identify the repos, but git projects
# can have duplicated names, so these projects are not actually hosted on multiple repos
temp = data.groupby(['project', 'host_type']).count()['status']
non_multi_repo_projects = [x for (x,y) in temp[temp > 1].index]
non_multi_repo_projects

['Python', 'algorithms', 'dataset', 'models', 'seq2seq', 'sshuttle', 'tqdm']

In [15]:
temp = data.groupby(['project']).count()['status']
temp[temp == 1] = 0
temp[temp > 1] = 1
# fix errors
temp[non_multi_repo_projects] = 0
temp[temp == 1].count(), temp[temp == 0].count()
multi_repo = []
for i, row in data.iterrows():
    multi_repo.append(temp[row['project']])
data['multi_repo'] = multi_repo
data[data['multi_repo'] == 1]


,url,host_type,major_releases,author_count,duration,rev_count,project,status,multi_repo
6,deb://Debian/packages/ansible,deb,0.0,3.0,41.0,496.0,ansible,0,1
18,deb://Debian/packages/asciinema,deb,0.0,1.0,49.0,333.0,asciinema,0,1
21,deb://Debian/packages/autojump,deb,0.0,1.0,58.0,64.0,autojump,0,1
27,deb://Debian/packages/beets,deb,0.0,4.0,68.0,314.0,beets,0,1
44,deb://Debian/packages/buildbot,deb,0.0,3.0,73.0,264.0,buildbot,0,1
...,...,...,...,...,...,...,...,...,...
2058,https://pypi.org/project/troposphere/,pypi,0.0,1.0,62.0,10.0,troposphere,0,1
2065,https://pypi.org/project/uvloop/,pypi,0.0,2.0,21.0,9.0,uvloop,0,1
2068,https://pypi.org/project/virtualenv/,pypi,0.0,2.0,119.0,2.0,virtualenv,0,1
2071,https://pypi.org/project/watchdog/,pypi,0.0,2.0,52.0,1.0,watchdog,0,1


Before we save it to a CSV lets check it out first

In [16]:
data

,url,host_type,major_releases,author_count,duration,rev_count,project,status,multi_repo
0,deb://Debian/packages/accerciser,deb,0.0,4.0,73.0,358.0,accerciser,0,0
1,deb://Debian/packages/agtl,deb,0.0,2.0,59.0,NaN,agtl,1,0
2,deb://Debian/packages/ajaxterm,deb,0.0,2.0,49.0,NaN,ajaxterm,1,0
3,deb://Debian/packages/alacarte,deb,0.0,3.0,71.0,347.0,alacarte,0,0
4,deb://Debian/packages/angrydd,deb,0.0,3.0,98.0,NaN,angrydd,1,0
...,...,...,...,...,...,...,...,...,...
2080,https://pypi.org/project/youtube_dl/,pypi,0.0,2.0,64.0,53.0,youtube_dl,0,0
2081,https://pypi.org/project/zc.lockfile/,pypi,0.0,2.0,116.0,1.0,zc.lockfile,0,0
2082,https://pypi.org/project/zope.component/,pypi,0.0,3.0,138.0,NaN,zope.component,1,0
2083,https://pypi.org/project/zope.deprecation/,pypi,0.0,2.0,136.0,NaN,zope.deprecation,1,0


We don't really need the url or rev_count so lets drop those

In [17]:
data = data.drop(['url', 'rev_count'], axis = 1)

Now lets check to see if we have any NA values

In [18]:
data.isna().sum()

host_type         0
major_releases    0
author_count      0
duration          0
project           0
status            0
multi_repo        0
dtype: int64

Okay that's good, no NA values.  
Lets change the order of the columns now so they make a little more sense

In [19]:
data[['project', 'status', 'major_releases', 'host_type', 'author_count', 'multi_repo', 'duration']]

,project,status,major_releases,host_type,author_count,multi_repo,duration
0,accerciser,0,0.0,deb,4.0,0,73.0
1,agtl,1,0.0,deb,2.0,0,59.0
2,ajaxterm,1,0.0,deb,2.0,0,49.0
3,alacarte,0,0.0,deb,3.0,0,71.0
4,angrydd,1,0.0,deb,3.0,0,98.0
...,...,...,...,...,...,...,...
2080,youtube_dl,0,0.0,pypi,2.0,0,64.0
2081,zc.lockfile,0,0.0,pypi,2.0,0,116.0
2082,zope.component,1,0.0,pypi,3.0,0,138.0
2083,zope.deprecation,1,0.0,pypi,2.0,0,136.0


In [20]:
data.to_csv('data.csv')